## Deep dive into SwapDeposit()

In [1]:
import os
import copy
import numpy as np
import pandas as pd
import time
import datetime
import math as mth
import pytest
from termcolor import colored
import matplotlib.pyplot as plt

In [2]:
from uniswappy import *

In [3]:
user_nm = 'user0'
eth_amount = 1000
tkn_amount = 100000

#### SwapDeposit (incl. fee):

* A ```SwapDeposit``` is where a certain amount of a specific token is deposited into the LP under one operation
* Includes two steps:
    * (step 1) perform approx. 50% swap for opposing token
    * (step 2) using amt from step 1, perform 1:1 deposit
* A portion of the incoming funds are swapped to achieve equal portions of both assets
* These portions are then deposited into the LP
* To ensure all the funds are deposited, we must determine the portion ($\alpha$) of $s_{in}$ that must first get swapped

> $ \Delta x =\Delta x_{swap}  + \Delta x_{deposit}$

> $ \Delta x =\alpha \Delta x   + \Delta x_{deposit}$


#### Follows this system of equations:

> (Eq. 1) $\Delta y = \frac{997 y \alpha \Delta x }{1000x + 997\alpha\Delta x }$

> (Eq. 2) $\Delta x = \alpha\Delta x   + \frac{\Delta y(x + \alpha\Delta x )}{y - \Delta y}$

where 
* $\Delta x$ -> amt token in 
* $\Delta y$ -> amt opposing token out after swap
* $\alpha$ -> portion of $\Delta x$ swapped in
* x -> reserve0
* y -> reserve1

### Let's highlight why the above considerations are important ...

In [4]:
tkn = ERC20("TKN", "0x111")
eth = ERC20("ETH", "0x09")
factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.create_exchange(eth, tkn, symbol="LP", address="0x011")
lp.add_liquidity(user_nm, eth_amount, tkn_amount, eth_amount, tkn_amount)

print('***\nInitial LP\n***')
lp.info()

s_in = 100
alpha = 0.5
s_out = Swap().apply(lp, eth, user_nm, alpha*s_in)

print('***\nLP post step 1\n***')
lp.info()
    
balance0 = alpha*s_in
balance1 = lp.quote(balance0, lp.reserve0, lp.reserve1)
lp.add_liquidity(user_nm, balance0, balance1, balance0, balance1) 

print('***\nLP post step 2\n***')
lp.info() 


print('Given {} initial ETH:'.format(s_in))
print('  (step 1) {} ETH must first get swapped for {} TKN'.format(alpha*s_in, s_out))
print('  (step 2) The received TKN gets deposited along with the remaining {} ETH'.format(balance0))

print('\nTotal deposited is {:.6f} + {:.6f} = {:.6f} ETH:'.format(alpha*s_in,  balance0, alpha*s_in + balance0))
print('However, we lost {} TKN which need to be considered when using a 50/50 split'.format(colored(str(tkn_amount-lp.reserve1), 'red', attrs=['bold'])))

***
Initial LP
***
Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 1000 | TKN = 100000
Liquidity: 10000.0 

***
LP post step 1
***
Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 1050.0 | TKN = 95251.7026241844
Liquidity: 10000.0 

***
LP post step 2
***
Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 1100.0 | TKN = 99787.4979872408
Liquidity: 10476.190476190477 

Given 100 initial ETH:
  (step 1) 50.0 ETH must first get swapped for 4748.297375815593 TKN
  (step 2) The received TKN gets deposited along with the remaining 50.0 ETH

Total deposited is 50.000000 + 50.000000 = 100.000000 ETH:
However, we lost 212.50201275919972 TKN which need to be considered when using a 50/50 split


### Let's now address this problem ...

In [5]:
tkn = ERC20("TKN", "0x111")
eth = ERC20("ETH", "0x09")
factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.create_exchange(eth, tkn, symbol="LP", address="0x011")
lp.add_liquidity(user_nm, eth_amount, tkn_amount, eth_amount, tkn_amount)

s_in = 100
alpha = 0.5
y = lp.reserve1
x = lp.reserve0

#### Plug above into equation (1), and see how many TKN we get when 50% of ETH is swapped in for step (1)

In [6]:
s_out = (997*alpha*s_in*y)/(1000*x + 997*alpha*s_in)
print('For {} ETH, we get {:.2f} TKN with a 50% portion'.format(s_in, s_out))

For 100 ETH, we get 4748.30 TKN with a 50% portion


#### Now, lets check how many ETH gets SwapDeposited in when 50% of ETH is swapped in for step (1)

In [7]:
a1_out = alpha*s_in + s_out*(x + alpha*s_in)/(y - s_out)
print('Instead of {} ETH, we get {:.2f} ETH under a 50% portion'.format(s_in, a1_out))

Instead of 100 ETH, we get 102.34 ETH under a 50% portion


We can see that there is an imbalance in the system under a 50% distribution for step (1); 
* we need to solve the system above for $\alpha$ to get the proper distribution
* plug (1) into (2) and we get:

$\Delta x = \alpha\Delta x + \left(\frac{997 y\alpha\Delta x}{1000x + 997\alpha\Delta x} \right) \left(\frac{ x + \alpha\Delta x}{y - \frac{997 y \alpha \Delta x}{1000x + 997\alpha\Delta x}} \right)$

reduces to: 

$\alpha^2 \frac{997 \Delta x^2}{1000x} + \alpha\frac{1997\Delta x}{1000} - \Delta x = 0$

#### Now, solve for $\alpha$, and we can calculate the correct distribution using ```calc_deposit_dist``` 

In [8]:
def calc_deposit_portion(lp, token_in, dx):
        
    if(token_in.token_name == lp.token0):
        tkn_supply = lp.reserve0
    else:    
        tkn_supply = lp.reserve1
            
    a = 997*(dx**2)/(1000*tkn_supply)
    b = dx*(1997/1000)
    c = -dx

    alpha = -(b - mth.sqrt(b*b - 4*a*c)) / (2*a)
    return alpha 

In [9]:
alpha = calc_deposit_portion(lp, eth, s_in)
print('The correct swap distrbution (for step 1) is {}'.format(alpha))

The correct swap distrbution (for step 1) is 0.4888217399419355


#### Now, check against our reduced quadratic, and we should expect to get 0

In [10]:
997*(alpha**2)*(s_in**2)/(1000*x) + alpha*s_in*(1997/1000) - s_in

-5.684341886080802e-14

#### Finally, lets run through the steps to a ```SwapDeposit``` and compare above

In [11]:
tkn = ERC20("TKN", "0x111")
eth = ERC20("ETH", "0x09")
factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.create_exchange(eth, tkn, symbol="LP", address="0x011")
lp.add_liquidity(user_nm, eth_amount, tkn_amount, eth_amount, tkn_amount)

print('***\nInitial LP\n***')
lp.info()

s_in = 100
alpha = calc_deposit_portion(lp, eth, s_in)
s_out = Swap().apply(lp, eth, user_nm, alpha*s_in)

print('***\nLP post step 1\n***')
lp.info()
    
balance1 = s_out
balance0 = lp.quote(s_out, lp.reserve1, lp.reserve0)
lp.add_liquidity(user_nm, balance0, balance1, balance0, balance1) 

print('***\nLP post step 2\n***')
lp.info() 


print('Given {} initial ETH:'.format(s_in))
print('  (step 1) {} ETH must first get swapped for {} TKN'.format(alpha*s_in, s_out))
print('  (step 2) The received TKN gets deposited along with the remaining {} ETH'.format(balance0))

print('\nTotal deposited is {:.6f} + {:.6f} = {:.6f} ETH:'.format(alpha*s_in,  balance0, alpha*s_in + balance0))

***
Initial LP
***
Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 1000 | TKN = 100000
Liquidity: 10000.0 

***
LP post step 1
***
Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 1048.8821739941936 | TKN = 95352.92490856306
Liquidity: 10000.0 

***
LP post step 2
***
Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 1100.0 | TKN = 100000.0
Liquidity: 10487.35527472211 

Given 100 initial ETH:
  (step 1) 48.88217399419355 ETH must first get swapped for 4647.075091436944 TKN
  (step 2) The received TKN gets deposited along with the remaining 51.11782600580638 ETH

Total deposited is 48.882174 + 51.117826 = 100.000000 ETH:


#### Let's check the opposing token

In [12]:
tkn = ERC20("TKN", "0x111")
eth = ERC20("ETH", "0x09")
factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.create_exchange(eth, tkn, symbol="LP", address="0x011")
lp.add_liquidity(user_nm, eth_amount, tkn_amount, eth_amount, tkn_amount)

print('***\nInitial LP\n***')
lp.info()

s_in = 1000
alpha = calc_deposit_portion(lp, tkn, s_in)
s_out = Swap().apply(lp, tkn, user_nm, alpha*s_in)

print('***\nLP post step 1\n***')
lp.info()
   
balance1 = lp.quote(s_out, lp.reserve0, lp.reserve1)    
balance0 = s_out
lp.add_liquidity(user_nm, balance0, balance1, balance0, balance1) 

print('***\nLP post step 2\n***')
lp.info() 


print('Given {} initial ETH:'.format(s_in))
print('  (step 1) {} ETH must first get swapped for {} TKN'.format(alpha*s_in, s_out))
print('  (step 2) The received TKN gets deposited along with the remaining {} ETH'.format(balance0))

print('\nTotal deposited is {:.6f} + {:.6f} = {:.6f} TKN:'.format(alpha*s_in,  balance1, alpha*s_in + balance1))

***
Initial LP
***
Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 1000 | TKN = 100000
Liquidity: 10000.0 

***
LP post step 1
***
Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 995.0446086357448 | TKN = 100499.50547221022
Liquidity: 10000.0 

***
LP post step 2
***
Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 1000.0 | TKN = 101000.0
Liquidity: 10049.80069557936 

Given 1000 initial ETH:
  (step 1) 499.5054722102269 ETH must first get swapped for 4.955391364255177 TKN
  (step 2) The received TKN gets deposited along with the remaining 4.955391364255177 ETH

Total deposited is 499.505472 + 500.494528 = 1000.000000 TKN:


#### Finally, let's check when our solution is integrated into ```SwapDeposit```

In [13]:
tkn = ERC20("TKN", "0x111")
eth = ERC20("ETH", "0x09")
factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.create_exchange(eth, tkn, symbol="LP", address="0x011")
lp.add_liquidity(user_nm, eth_amount, tkn_amount, eth_amount, tkn_amount)
lp.info()

s_in = 100
dep = SwapDeposit().apply(lp, eth, user_nm, s_in)
lp.info()

Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 1000 | TKN = 100000
Liquidity: 10000.0 

Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 1100.0 | TKN = 100000.0
Liquidity: 10487.35527472211 



In [14]:
tkn = ERC20("TKN", "0x111")
eth = ERC20("ETH", "0x09")
factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.create_exchange(eth, tkn, symbol="LP", address="0x011")
lp.add_liquidity(user_nm, eth_amount, tkn_amount, eth_amount, tkn_amount)
lp.info()

Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 1000 | TKN = 100000
Liquidity: 10000.0 



In [15]:
tkn = ERC20("TKN", "0x111")
eth = ERC20("ETH", "0x09")
factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.create_exchange(eth, tkn, symbol="LP", address="0x011")
lp.add_liquidity(user_nm, eth_amount, tkn_amount, eth_amount, tkn_amount)
lp.info()

dep = SwapDeposit().apply(lp, tkn, user_nm, 100)
lp.info()

out = WithdrawSwap().apply(lp, tkn, user_nm, 100)
lp.info()

Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 1000 | TKN = 100000
Liquidity: 10000.0 

Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 1000.0 | TKN = 100100.0
Liquidity: 10004.991241237401 

Exchange ETH/TKN (LP)
Coins: ETH/TKN
Reserves: ETH = 1000.0 | TKN = 99999.99999999997
Liquidity: 9999.984988734976 

